In [1]:
import pickle
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
with open('./graphs.pkl', 'rb') as f:
    graphs = pickle.load(f)

In [3]:
for n, data in graphs[0].nodes(data=True):
    if data['node_type'] != 'concept':
        continue
    print(n, data)
    break

c_7261 {'node_type': 'concept', 'node_feature': array([ 1.49000000e+02,  5.56243844e-02, -1.32697016e-01,  1.80167049e-01,
       -8.54894668e-02,  1.19979054e-01,  1.53961644e-01, -5.16535603e-02,
       -1.38601307e-02, -7.87414759e-02,  3.96418236e-02,  1.13002896e-01,
        7.74992630e-02,  8.91781598e-03, -1.92961782e-01, -2.27575675e-02,
       -1.22387916e-01, -1.02404319e-01, -6.29378334e-02, -7.91537315e-02,
        6.92976937e-02,  1.20116279e-01,  4.12906967e-02, -3.20180416e-01,
        1.54964238e-01, -9.94347557e-02, -1.53273270e-02,  8.92051086e-02,
       -5.03597781e-02,  1.05510265e-01, -5.62509522e-05,  6.90224171e-02,
        2.94789709e-02,  6.50980398e-02,  5.58857024e-01,  1.00221947e-01,
       -2.15795830e-01, -6.94271028e-02,  1.14316893e+00, -4.50918555e-01,
        1.35639966e-01, -2.68275104e-02, -7.06465095e-02, -2.14624301e-01,
       -9.21650082e-02, -3.63600254e-02, -5.27555421e-02,  6.69585839e-02,
       -6.93681762e-02,  2.97508240e+00, -1.13852121

In [7]:
import pandas as pd

def get_dataframe(graph):
    # Define the data types for each column
    dtypes = {'node_id': 'str',
              'node_type': 'str', 
              'event_date': 'float32',  # Adjust this dtype based on the actual data type of your date
              'article_count': 'float32',  # Assuming article count is numeric
              'prediction': 'float32'}  # Assuming prediction is numeric

    df = pd.DataFrame(columns=dtypes.keys())
    embeddings = np.array([])
    for node, data in graph.nodes(data=True):
        node_type = data['node_type']
        embedding = data['embedding']
        embeddings = np.append(embeddings, embedding, axis=0)
        
        temp_dict = {'node_id': node,
                     'node_type': node_type, 
                     'event_date': None, 
                     'article_count': None, 
                     'prediction': None}
        
        if node_type == 'concept':
            df = pd.concat([df, pd.DataFrame([temp_dict], columns=df.columns)], ignore_index=True)
            continue
        
        features = data['node_feature']
        temp_dict['event_date'] = features[0] * 16600
        
        if node_type == 'event':
            temp_dict['article_count'] = features[1]
        else:
            temp_dict['article_count'] = data['node_target'][0]
            temp_dict['prediction'] = data['prediction'][0]
            
        df = pd.concat([df, pd.DataFrame([temp_dict], columns=df.columns)], ignore_index=True)
    # Convert columns to their respective data types
    for col, dtype in dtypes.items():
        df[col] = df[col].astype(dtype)
        
    return df, embeddings


In [8]:
df, embeds = get_dataframe(graphs[0])

/var/folders/t_/181rjdy93vg4rlj82l3c36_40000gn/T/ipykernel_68340/2897005210.py:37: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([temp_dict], columns=df.columns)], ignore_index=True)
/var/folders/t_/181rjdy93vg4rlj82l3c36_40000gn/T/ipykernel_68340/2897005210.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([temp_dict], columns=df.columns)], ignore_index=True)


In [9]:
df.head()

,node_id,node_type,event_date,article_count,prediction
0,e_2765400,event,215800.0,0.995000,NaN
1,e_2150458,event,514600.0,0.990241,NaN
2,e_1908730,event,597600.0,0.988313,NaN
3,e_2794829,event,564400.0,0.995361,NaN
4,e_1664256,event,365200.0,0.986566,NaN


In [10]:
# save df to pickle
with open('./df_0.pkl', 'wb') as f:
    pickle.dump(df, f)

In [11]:
# save embeddings to pickle
with open('./embeds_0.pkl', 'wb') as f:
    pickle.dump(embeds, f)

In [15]:
embeds = embeds.reshape(-1, 768)

ValueError: cannot reshape array of size 6077940 into shape (768)

In [14]:
import umap

reducer = umap.UMAP()
embedding = reducer.fit_transform(embeds.reshape(-1, 768))

AttributeError: module 'umap' has no attribute 'UMAP'

In [22]:
e = []
for node, data in graphs[0].nodes(data=True):
    e.append(data['embedding'])

In [24]:
e = np.array(e)

In [25]:
e.shape

(101299, 60)

In [ ]:
import umap.umap_ as umap

reducer = umap.UMAP()
embedding = reducer.fit_transform(e)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
